In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
import matplotlib.pyplot as plt

In [2]:
CHD =fetch_california_housing()

In [3]:
df = pd.DataFrame(data=CHD.data,columns=CHD.feature_names)
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [4]:
x_data = CHD.data
y = CHD['target']

print(x_data.shape,y.shape)

(20640, 8) (20640,)


In [5]:
X_train, X_eval,y_train,y_eval=train_test_split(x_data,y,test_size=0.3,random_state=101)

scaler_model = StandardScaler()
X_train = scaler_model.fit_transform(X_train)
X_eval   = scaler_model.transform(X_eval)

In [6]:
def DNN_Regressor(hidden_units,n_features):
    Inputs = tf.keras.layers.Input(shape=n_features)
    c = 0
    for i in hidden_units:
        if c==0:
            Hidden = tf.keras.layers.Dense(i,activation='relu')(Inputs)
        else:
            Hidden = tf.keras.layers.Dense(i,activation='relu')(Hidden)
        c =+1
    Out = tf.keras.layers.Dense(1,activation='relu')(Hidden)

    return tf.keras.Model(inputs=Inputs, outputs=Out)


In [7]:
# Hidden units son las neuronas de las capas ocultas y se puede modificar tanto número de neuronas como el número de capas
# asímismo los parámetros de la red como el optimizador y el learning rate
model = DNN_Regressor(hidden_units=[6,10,6],n_features = X_train.shape[1])
opt = tf.keras.optimizers.Adagrad(learning_rate=1e-1)
model.compile(optimizer=opt,loss='mse',metrics=['mse','mae'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 54        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                70        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 197
Trainable params: 197
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Se puede cambiar el tamaño del batch y las epocas 
history = model.fit(X_train,y_train,batch_size=32,epochs=100,validation_split=0.2,verbose=1)

Epoch 1/100
362/362 [==============================] - 8s 11ms/step - loss: 0.8970 - mse: 0.8970 - mae: 0.6575 - val_loss: 0.3821 - val_mse: 0.3821 - val_mae: 0.4411
Epoch 2/100
362/362 [==============================] - 1s 2ms/step - loss: 0.3831 - mse: 0.3831 - mae: 0.4438 - val_loss: 0.3724 - val_mse: 0.3724 - val_mae: 0.4252
Epoch 3/100
362/362 [==============================] - 1s 1ms/step - loss: 0.3609 - mse: 0.3609 - mae: 0.4307 - val_loss: 0.3599 - val_mse: 0.3599 - val_mae: 0.4232
Epoch 4/100
362/362 [==============================] - 1s 1ms/step - loss: 0.3482 - mse: 0.3482 - mae: 0.4223 - val_loss: 0.3409 - val_mse: 0.3409 - val_mae: 0.4244
Epoch 5/100
362/362 [==============================] - 1s 1ms/step - loss: 0.3400 - mse: 0.3400 - mae: 0.4201 - val_loss: 0.3414 - val_mse: 0.3414 - val_mae: 0.4190
Epoch 6/100
362/362 [==============================] - 1s 1ms/step - loss: 0.3518 - mse: 0.3518 - mae: 0.4220 - val_loss: 0.3505 - val_mse: 0.3505 - val_mae: 0.4300
Epoch 7/1

In [9]:
model.predict(X_eval)

array([[3.8818796],
       [5.242335 ],
       [1.2316169],
       ...,
       [1.2915999],
       [0.8011545],
       [2.5390742]], dtype=float32)